In [8]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from patsy import dmatrices
from sklearn import tree
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
%pylab inline
import warnings
warnings.filterwarnings('ignore')

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [34]:
df = pd.read_csv('olympics_new.csv')
df.head()

,country_name,country_3_letter_code,year,medal_count,gold_count,silver_count,bronze_count,game_location,game_season,GDP,population,region,area_sqkm,coastline_coast_area_ratio,GDP_per_capita,host_country_status
0,Austria,AUT,1992,21,6,7,8,France,Winter,1.950781e+11,7840709.0,WESTERN EUROPE,83870,0.00,24880.164118,0
1,Canada,CAN,1992,7,2,3,2,France,Winter,5.943761e+11,28371264.0,NORTHERN AMERICA,9984670,2.02,20949.933265,0
2,Czechia,TCH,1992,3,0,0,3,France,Winter,3.480501e+10,10319123.0,EASTERN EUROPE,78866,0.00,3372.865430,0
3,Finland,FIN,1992,7,3,1,3,France,Winter,1.125325e+11,5041992.0,WESTERN EUROPE,338145,0.37,22319.059460,0
4,France,FRA,1992,9,3,5,1,France,Winter,1.401466e+12,58849943.0,WESTERN EUROPE,547030,0.63,23814.227368,1


In [36]:
df['GDP_growth_rate']= df.sort_values(['country_name', 'year']).groupby('country_name')['GDP'].pct_change() * 100
df['population_density'] = df['population'] / df['area_sqkm']
df.head()

,country_name,country_3_letter_code,year,medal_count,gold_count,silver_count,bronze_count,game_location,game_season,GDP,population,region,area_sqkm,coastline_coast_area_ratio,GDP_per_capita,host_country_status,GDP_growth_rate,population_density
0,Austria,AUT,1992,21,6,7,8,France,Winter,1.950781e+11,7840709.0,WESTERN EUROPE,83870,0.00,24880.164118,0,46.301942,93.486455
1,Canada,CAN,1992,7,2,3,2,France,Winter,5.943761e+11,28371264.0,NORTHERN AMERICA,9984670,2.02,20949.933265,0,16.686259,2.841482
2,Czechia,TCH,1992,3,0,0,3,France,Winter,3.480501e+10,10319123.0,EASTERN EUROPE,78866,0.00,3372.865430,0,NaN,130.843748
3,Finland,FIN,1992,7,3,1,3,France,Winter,1.125325e+11,5041992.0,WESTERN EUROPE,338145,0.37,22319.059460,0,3.185000,14.910739
4,France,FRA,1992,9,3,5,1,France,Winter,1.401466e+12,58849943.0,WESTERN EUROPE,547030,0.63,23814.227368,1,37.554104,107.580833


In [38]:
#removed 'game_location'
selected_cols = ['GDP', 'population', 'GDP_per_capita', 'population_density', 'host_country_status', 'region', 'GDP_growth_rate', 'coastline_coast_area_ratio', 'game_season', 'medal_count']
df = df[selected_cols]
df.head()

,GDP,population,GDP_per_capita,population_density,host_country_status,region,GDP_growth_rate,coastline_coast_area_ratio,game_season,medal_count
0,1.950781e+11,7840709.0,24880.164118,93.486455,0,WESTERN EUROPE,46.301942,0.00,Winter,21
1,5.943761e+11,28371264.0,20949.933265,2.841482,0,NORTHERN AMERICA,16.686259,2.02,Winter,7
2,3.480501e+10,10319123.0,3372.865430,130.843748,0,EASTERN EUROPE,NaN,0.00,Winter,3
3,1.125325e+11,5041992.0,22319.059460,14.910739,0,WESTERN EUROPE,3.185000,0.37,Winter,7
4,1.401466e+12,58849943.0,23814.227368,107.580833,1,WESTERN EUROPE,37.554104,0.63,Winter,9


In [40]:
#olympic season split dfs
summer_mask = df['game_season'] == 'Summer'

summer_df = df[summer_mask]
winter_df = df[~summer_mask]

summer_df.drop(columns = 'game_season', inplace = True)
winter_df.drop(columns = 'game_season', inplace = True)

In [44]:
formula = 'medal_count ~ ' + ' + '.join(['Q("'+x+'")' \
                                    for x in summer_df.columns \
                                    if x != 'medal_count'])
print(formula)
summer_df

medal_count ~ Q("GDP") + Q("population") + Q("GDP_per_capita") + Q("population_density") + Q("host_country_status") + Q("region") + Q("GDP_growth_rate") + Q("coastline_coast_area_ratio")


,GDP,population,GDP_per_capita,population_density,host_country_status,region,GDP_growth_rate,coastline_coast_area_ratio,medal_count
19,1.646579e+11,38668796.0,4258.160261,13.975545,0,LATIN AMER. & CARIB,-42.063421,0.18,6
20,6.143264e+11,19932722.0,30819.993487,2.593094,0,OCEANIA,55.300368,0.34,50
21,3.014576e+11,8171966.0,36889.233514,97.436103,0,WESTERN EUROPE,40.608573,0.00,7
22,8.680410e+09,8306500.0,1045.014165,95.918014,0,C.W. OF IND. STATES,64.631905,0.00,5
23,9.055290e+09,343089.0,26393.413954,24.611836,0,LATIN AMER. & CARIB,12.119404,25.41,2
...,...,...,...,...,...,...,...,...,...
1131,4.561057e+10,6250438.0,7297.180042,12.805650,0,C.W. OF IND. STATES,NaN,0.00,1
1132,3.760543e+10,44404611.0,846.881199,188.123246,0,SUB-SAHARAN AFRICA,37.718985,0.00,4
1133,1.566177e+11,44132049.0,3548.843150,73.102616,0,C.W. OF IND. STATES,19.654992,0.46,19
1134,2.106047e+13,331511512.0,63528.634303,34.419796,0,NORTHERN AMERICA,2.568620,0.21,113


### Summer Olympics

In [67]:
summer_Y, summer_X = dmatrices(formula, summer_df, return_type='dataframe')
summer_y = summer_Y['medal_count'].values

In [68]:
summer_X_train, summer_X_test, summer_y_train, summer_y_test = train_test_split(summer_X, summer_y,
                                                    test_size=0.2,  # 20% of the data will be used as the test set
                                                    random_state=1)

In [70]:
# All the models we want to test out

#single decision tree
summer_clf = tree.DecisionTreeRegressor(random_state = 1)
summer_clf.fit(summer_X_train, summer_y_train)

#bagging
summer_bagging = BaggingRegressor(tree.DecisionTreeRegressor(), random_state=1)
summer_bagging.fit(summer_X_train, summer_y_train)

#random forest
summer_rf = RandomForestRegressor(max_features = 'sqrt', random_state=1)
summer_rf.fit(summer_X_train, summer_y_train)

#gradient boosting
summer_gb = GradientBoostingRegressor(random_state=1)
summer_gb.fit(summer_X_train, summer_y_train)


GradientBoostingRegressor(random_state=1)

In [53]:
# Define a function to evaluate models
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    return {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R²': r2}

In [55]:
summer_clf_results = evaluate_model(summer_clf, summer_X_test, summer_y_test)
summer_bagging_results = evaluate_model(summer_bagging, summer_X_test, summer_y_test)
summer_rf_results = evaluate_model(summer_rf, summer_X_test, summer_y_test)
summer_gb_results = evaluate_model(summer_gb, summer_X_test, summer_y_test)

# Display the results
base_summer_results_df = pd.DataFrame([summer_clf_results, summer_bagging_results, summer_rf_results, summer_gb_results],
                          index=['Decision Tree', 'Bagging', 'Random Forest', 'Gradient Boosting'])
print(base_summer_results_df)

                        MAE         MSE       RMSE        R²
Decision Tree      7.785714  224.551948  14.985057  0.322729
Bagging            5.201948   85.332792   9.237575  0.742628
Random Forest      4.674238   52.975255   7.278410  0.840221
Gradient Boosting  5.013585   66.179258   8.135063  0.800397


In [57]:
#grid search Bagging

bagging_param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_samples': [0.5, 0.7, 1.0],
    'max_features': [0.5, 0.7, 1.0],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
}

summer_bagging_cv = BaggingRegressor(random_state = 1)
# Example of setting up the grid search
summer_bagging_grid_search = GridSearchCV(estimator=summer_bagging_cv, param_grid = bagging_param_grid, scoring='neg_mean_squared_error', n_jobs=-1)

summer_bagging_grid_search.fit(summer_X_train, summer_y_train)

GridSearchCV(estimator=BaggingRegressor(random_state=1), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'bootstrap_features': [True, False],
                         'max_features': [0.5, 0.7, 1.0],
                         'max_samples': [0.5, 0.7, 1.0],
                         'n_estimators': [10, 50, 100, 200]},
             scoring='neg_mean_squared_error')

In [58]:
summer_bagging_best_params = summer_bagging_grid_search.best_params_

print(f'Best Parameters: {summer_bagging_best_params}')

Best Parameters: {'bootstrap': False, 'bootstrap_features': False, 'max_features': 0.7, 'max_samples': 0.7, 'n_estimators': 200}


In [59]:
#grid search Random Forest

rf_param_grid = {
    'n_estimators': [100, 200, 250],
    'max_depth': [3, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt'],
    'bootstrap': [True, False]
}

summer_rf_cv = RandomForestRegressor(random_state = 1)

summer_rf_grid_search = GridSearchCV(estimator = summer_rf_cv, param_grid = rf_param_grid, scoring='neg_mean_squared_error', n_jobs=-1)

summer_rf_grid_search.fit(summer_X_train, summer_y_train)

GridSearchCV(estimator=RandomForestRegressor(random_state=1), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [3, 5, 10, 20], 'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 250]},
             scoring='neg_mean_squared_error')

In [60]:
summer_rf_best_params = summer_rf_grid_search.best_params_

print(f'Best Parameters: {summer_rf_best_params}')

Best Parameters: {'bootstrap': True, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [61]:
#grid search Gradient Boosting

gb_param_grid = {
    'n_estimators': [100, 200, 250],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 1.0],
    'max_features': ['sqrt']
}

summer_gb_cv = GradientBoostingRegressor(random_state = 1)

summer_gb_grid_search = GridSearchCV(estimator = summer_gb_cv, param_grid = gb_param_grid, scoring='neg_mean_squared_error', n_jobs=-1)

summer_gb_grid_search.fit(summer_X_train, summer_y_train)

GridSearchCV(estimator=GradientBoostingRegressor(random_state=1), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1], 'max_depth': [3, 5, 10],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200, 250],
                         'subsample': [0.8, 1.0]},
             scoring='neg_mean_squared_error')

In [62]:
summer_gb_best_params = summer_gb_grid_search.best_params_

print(f'Best Parameters: {summer_gb_best_params}')

Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 250, 'subsample': 0.8}


In [63]:
#Grid search models

#bagging
grid_summer_bagging = BaggingRegressor(n_estimators = 200, max_samples = 0.7, max_features = 0.7,
                                      bootstrap = False, bootstrap_features = False,  random_state = 1)
grid_summer_bagging.fit(summer_X_train, summer_y_train)

#random forest
grid_summer_rf = RandomForestRegressor(n_estimators = 100, max_depth = 20, min_samples_leaf = 1, min_samples_split = 2,
                                           bootstrap = True, max_features = 'sqrt', random_state=1)
grid_summer_rf.fit(summer_X_train, summer_y_train)

#gradient boosting
grid_summer_gb = GradientBoostingRegressor(n_estimators = 250, max_depth = 5, min_samples_leaf = 2, min_samples_split = 2,
                                                learning_rate = 0.1, max_features = 'sqrt', subsample = 0.8, random_state=1)
grid_summer_gb.fit(summer_X_train, summer_y_train)

GradientBoostingRegressor(max_depth=5, max_features='sqrt', min_samples_leaf=2,
                          n_estimators=250, random_state=1, subsample=0.8)

In [64]:
grid_summer_bagging_results = evaluate_model(grid_summer_bagging, summer_X_test, summer_y_test)
grid_summer_rf_results = evaluate_model(grid_summer_rf, summer_X_test, summer_y_test)
grid_summer_gb_results = evaluate_model(grid_summer_gb, summer_X_test, summer_y_test)

# Display the results
summer_grid_results_df = pd.DataFrame([grid_summer_bagging_results, grid_summer_rf_results, grid_summer_gb_results],
                          index=['Bagging', 'Random Forest', 'Gradient Boosting'])
print(summer_grid_results_df)


                        MAE        MSE      RMSE        R²
Bagging            4.794478  57.540310  7.585533  0.826453
Random Forest      4.507012  49.951581  7.067643  0.849341
Gradient Boosting  4.538984  47.197012  6.870008  0.857649


In [123]:
##non-grid vs grid
summer_based_vs_grid_results_df = pd.DataFrame([summer_clf_results, summer_bagging_results, summer_rf_results, summer_gb_results,
                                         grid_summer_bagging_results, grid_summer_rf_results, grid_summer_gb_results],
                           index=['Decision Tree', 'Bagging', 'Random Forest', 'Gradient Boosting',
                                  'Optimized Bagging', 'Optimized Random Forest', 'Optimized Gradient Boosting'])

print(summer_based_vs_grid_results_df)

array([113.81950708])

### Winter Olympics

In [21]:
winter_Y, winter_X = dmatrices(formula, winter_df, return_type='dataframe')
winter_y = winter_Y['medal_count'].values

In [22]:
winter_X_train, winter_X_test, winter_y_train, winter_y_test = train_test_split(winter_X, winter_y,
                                                    test_size=0.2,  # 20% of the data will be used as the test set
                                                    random_state=1)

In [23]:
# All the models we want to test out

#single decision tree
winter_clf = tree.DecisionTreeRegressor(random_state = 1)
winter_clf.fit(winter_X_train, winter_y_train)

#bagging
winter_bagging = BaggingRegressor(tree.DecisionTreeRegressor(), random_state=1)
winter_bagging.fit(winter_X_train, winter_y_train)

#random forest
winter_rf = RandomForestRegressor(max_features = 'sqrt', random_state=1)
winter_rf.fit(winter_X_train, winter_y_train)

#gradient boosting
winter_gb = GradientBoostingRegressor(random_state=1)
winter_gb.fit(winter_X_train, winter_y_train)


GradientBoostingRegressor(random_state=1)

In [24]:
winter_clf_results = evaluate_model(winter_clf, winter_X_test, winter_y_test)
winter_bagging_results = evaluate_model(winter_bagging, winter_X_test, winter_y_test)
winter_rf_results = evaluate_model(winter_rf, winter_X_test, winter_y_test)
winter_gb_results = evaluate_model(winter_gb, winter_X_test, winter_y_test)

# Display the results
base_winter_results_df = pd.DataFrame([winter_clf_results, winter_bagging_results, winter_rf_results, winter_gb_results],
                          index=['Decision Tree', 'Bagging', 'Random Forest', 'Gradient Boosting'])
print(base_winter_results_df)

                        MAE        MSE      RMSE        R²
Decision Tree      5.224138  49.396552  7.028268  0.484955
Bagging            3.950000  30.017759  5.478846  0.687013
Random Forest      4.059540  31.426216  5.605909  0.672327
Gradient Boosting  4.152153  32.578276  5.707738  0.660315


In [25]:
#Winter Grid Search

#bagging
winter_bagging_cv = BaggingRegressor(random_state = 1)
# Example of setting up the grid search
winter_bagging_grid_search = GridSearchCV(estimator=winter_bagging_cv, param_grid = bagging_param_grid, scoring='neg_mean_squared_error', n_jobs=-1)

winter_bagging_grid_search.fit(winter_X_train, winter_y_train)

GridSearchCV(estimator=BaggingRegressor(random_state=1), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'bootstrap_features': [True, False],
                         'max_features': [0.5, 0.7, 1.0],
                         'max_samples': [0.5, 0.7, 1.0],
                         'n_estimators': [10, 50, 100, 200]},
             scoring='neg_mean_squared_error')

In [26]:
winter_bagging_best_params = winter_bagging_grid_search.best_params_

print(f'Best Parameters: {winter_bagging_best_params}')

Best Parameters: {'bootstrap': False, 'bootstrap_features': False, 'max_features': 1.0, 'max_samples': 0.7, 'n_estimators': 100}


In [27]:
#random forest
winter_rf_cv = RandomForestRegressor(random_state = 1)

winter_rf_grid_search = GridSearchCV(estimator = winter_rf_cv, param_grid = rf_param_grid, scoring='neg_mean_squared_error', n_jobs=-1)

winter_rf_grid_search.fit(winter_X_train, winter_y_train)

GridSearchCV(estimator=RandomForestRegressor(random_state=1), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [3, 5, 10, 20], 'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 250]},
             scoring='neg_mean_squared_error')

In [28]:
winter_rf_best_params = winter_rf_grid_search.best_params_

print(f'Best Parameters: {winter_rf_best_params}')

Best Parameters: {'bootstrap': False, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [29]:
#gradient boosting
winter_gb_cv = GradientBoostingRegressor(random_state = 1)

winter_gb_grid_search = GridSearchCV(estimator = winter_gb_cv, param_grid = gb_param_grid, scoring='neg_mean_squared_error', n_jobs=-1)

winter_gb_grid_search.fit(winter_X_train, winter_y_train)

GridSearchCV(estimator=GradientBoostingRegressor(random_state=1), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1], 'max_depth': [3, 5, 10],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200, 250],
                         'subsample': [0.8, 1.0]},
             scoring='neg_mean_squared_error')

In [30]:
winter_gb_best_params = winter_gb_grid_search.best_params_

print(f'Best Parameters: {winter_gb_best_params}')

Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 250, 'subsample': 0.8}


In [31]:
#Grid search models

#bagging
grid_winter_bagging = BaggingRegressor(n_estimators = 100, max_samples = 0.7, max_features = 1.0,
                                      bootstrap = False, bootstrap_features = False,  random_state = 1)
grid_winter_bagging.fit(winter_X_train, winter_y_train)

#random forest
grid_winter_rf = RandomForestRegressor(n_estimators = 200, max_depth = 20, min_samples_leaf = 1, min_samples_split = 2,
                                           bootstrap = False, max_features = 'sqrt', random_state=1)
grid_winter_rf.fit(winter_X_train, winter_y_train)

#gradient boosting
grid_winter_gb = GradientBoostingRegressor(n_estimators = 250, max_depth = 3, min_samples_leaf = 4, min_samples_split = 2,
                                                learning_rate = 0.1, max_features = 'sqrt', subsample = 0.8, random_state=1)
grid_winter_gb.fit(winter_X_train, winter_y_train)

GradientBoostingRegressor(max_features='sqrt', min_samples_leaf=4,
                          n_estimators=250, random_state=1, subsample=0.8)

In [32]:
grid_winter_bagging_results = evaluate_model(grid_winter_bagging, winter_X_test, winter_y_test)
grid_winter_rf_results = evaluate_model(grid_winter_rf, winter_X_test, winter_y_test)
grid_winter_gb_results = evaluate_model(grid_winter_gb, winter_X_test, winter_y_test)

# Display the results
winter_grid_results_df = pd.DataFrame([grid_winter_bagging_results, grid_winter_rf_results, grid_winter_gb_results],
                          index=['Bagging', 'Random Forest', 'Gradient Boosting'])
print(winter_grid_results_df)


                        MAE        MSE      RMSE        R²
Bagging            3.884483  30.226231  5.497839  0.684839
Random Forest      3.615690  27.948496  5.286634  0.708588
Gradient Boosting  3.675121  28.368045  5.326166  0.704214


In [33]:
##non-grid vs grid
winter_based_vs_grid_results_df = pd.DataFrame([winter_clf_results, winter_bagging_results, winter_rf_results, winter_gb_results,
                                         grid_winter_bagging_results, grid_winter_rf_results, grid_winter_gb_results],
                           index=['Decision Tree', 'Bagging', 'Random Forest', 'Gradient Boosting',
                                  'Optimized Bagging', 'Optimized Random Forest', 'Optimized Gradient Boosting'])

print(winter_based_vs_grid_results_df)

                                  MAE        MSE      RMSE        R²
Decision Tree                5.224138  49.396552  7.028268  0.484955
Bagging                      3.950000  30.017759  5.478846  0.687013
Random Forest                4.059540  31.426216  5.605909  0.672327
Gradient Boosting            4.152153  32.578276  5.707738  0.660315
Optimized Bagging            3.884483  30.226231  5.497839  0.684839
Optimized Random Forest      3.615690  27.948496  5.286634  0.708588
Optimized Gradient Boosting  3.675121  28.368045  5.326166  0.704214


In [34]:
print('SUMMER MODELS')
print(summer_based_vs_grid_results_df)
print()
print('WINTER MODELS')
print(winter_based_vs_grid_results_df)

SUMMER MODELS
                                  MAE         MSE       RMSE        R²
Decision Tree                7.785714  224.551948  14.985057  0.322729
Bagging                      5.201948   85.332792   9.237575  0.742628
Random Forest                4.674238   52.975255   7.278410  0.840221
Gradient Boosting            5.013585   66.179258   8.135063  0.800397
Optimized Bagging            4.794478   57.540310   7.585533  0.826453
Optimized Random Forest      4.507012   49.951581   7.067643  0.849341
Optimized Gradient Boosting  4.538984   47.197012   6.870008  0.857649

WINTER MODELS
                                  MAE        MSE      RMSE        R²
Decision Tree                5.224138  49.396552  7.028268  0.484955
Bagging                      3.950000  30.017759  5.478846  0.687013
Random Forest                4.059540  31.426216  5.605909  0.672327
Gradient Boosting            4.152153  32.578276  5.707738  0.660315
Optimized Bagging            3.884483  30.226231  5.497839

In [127]:
# USA predictions for 2024
usa2024 = pd.read_csv('USA2024.csv').head(1).drop(columns=['Unnamed: 0'])
grid_summer_gb.predict(usa2024)

array([113.81950708])